# 第9回講義 宿題

## 課題. 変分オートエンコーダ（VAE）でFasionMNISTの画像を生成せよ

## Homework

変分オートエンコーダ（VAE）により, FashionMNISTの画像を生成してみましょう.。

## ルール

- 訓練データはx_train、テストデータはx_testで与えられます。
- 下のセルで指定されているx_train以外の学習データは使わないでください。

## 目標値

NLL（負の対数尤度） 235

## 提出方法

- 2つのファイルを提出していただきます。
    1. テストデータ（x_test）について、VAEで生成した画像をcsvファイル（ファイル名：submission.csv）として保存し（1画像が1行に対応）、**Homeworkタブからchap09を選択して**提出してください。
    2. それに対応するpythonのコードをsubmission_code.pyとして保存し、**Homeworkタブからchap09 (code)を選択して**提出してください。 
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。


## 評価について

- 評価は生成画像のテストデータに対するNLL（負の対数尤度）で行います.  $-\sum_{i=1}^D x_i\log \hat{x}_i + (1-x_i)\log (1-\hat{x}_i)$
- 毎日夜24時にテストデータの一部に対するNLLでLeader Boardを更新します。
- 締切日の夜24時にテストデータ全体に対するNLLでLeader Boardを更新します。これを最終的な評価とします。

## サンプルコード

- 次のセルで指定されているx_trainのみを使って学習させてください.
- submission.csvの出力場所は適宜変更してください.

In [2]:
# このデータの読み込み部分は修正しないでください
import numpy as np
import pandas as pd

def load_mnist():

    # 学習データ
    x_train = np.load('/root/userspace/public/chap09/data/x_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap09/data/x_test.npy')

    x_train = (x_train.reshape(-1, 784) / 255).astype(np.float32)
    x_test = (x_test.reshape(-1, 784) / 255).astype(np.float32)

    return (x_train, x_test)

x_train, x_test = load_mnist()

## VAEの実装

In [4]:
#%%writefile /root/userspace/chap09/submission_code.py

import tensorflow as tf
import csv

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

rng = np.random.RandomState(1234)

### define layers ###
tf.reset_default_graph()
z_dim = 10

def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))

def encoder(x):
    # WRITE ME
    with tf.variable_scope('Encoder', reuse=tf.AUTO_REUSE):
        h1 = tf.layers.Dense(units=200, activation=tf.nn.relu)(x)
        h2 = tf.layers.Dense(units=200, activation=tf.nn.relu)(h1)
        mean = tf.layers.Dense(units=z_dim)(h2)
        var = tf.layers.Dense(units=z_dim, activation=tf.nn.softplus)(h2)
    return mean, var

def sampling_z(mean, var):
    # WRITE ME
    epsilon = tf.random_normal(shape=tf.shape(mean))
    z = mean + tf.sqrt(var) * epsilon
    return z

def decoder(z):
    # WRITE ME
    with tf.variable_scope('Decoder', reuse=tf.AUTO_REUSE):
        h3 = tf.layers.Dense(units=200, activation=tf.nn.relu)(z)
        h4 = tf.layers.Dense(units=200, activation=tf.nn.relu)(h3)
        y = tf.layers.Dense(units=784, activation=tf.nn.sigmoid)(h4)
    return y

def lower_bound(x):
    #Encode
    mean, var = encoder(x)
    KL = -0.5 * tf.reduce_mean(tf.reduce_sum(1 + tf_log(var) - mean**2 - var, axis=1))
    
    #Z
    z = sampling_z(mean, var)
    
    #Decode
    y = decoder(z)
    reconstruction = tf.reduce_mean(tf.reduce_sum(x * tf_log(y) + (1 - x) * tf_log(1 - y), axis=1))
    
    lower_bound = [-KL, reconstruction]
    
    return lower_bound


### training ###
#学習データと検証データに分割
x_train, x_valid = train_test_split(x_train, test_size=0.1)

x = tf.placeholder(tf.float32, [None, 784])
lower_bound = lower_bound(x)

cost = -tf.reduce_sum(lower_bound)
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(cost)

valid = tf.reduce_sum(lower_bound)

batch_size =100

n_batches = x_train.shape[0] // batch_size
n_epochs = 5

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for epoch in range(n_epochs):
    # WRITE ME
    rng.shuffle(x_train)
    lower_bound_all = []
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        _, lowerbound = sess.run([train, lower_bound], feed_dict={x: x_train[start:end]})
        lower_bound_all.append(lowerbound)
    lower_bound_all = np.mean(lower_bound_all, axis=0)
    lower_bound_valid = sess.run(valid, feed_dict={x: x_valid[0:100]})
    print('EPOCH:%d, Train Lower Bound:%lf, (%lf, %lf), Valid Lower Bound:%lf' %
             (epoch+1, np.sum(lower_bound_all), lower_bound_all[0], lower_bound_all[1], lower_bound_valid))
    
### sampling ###
x = tf.placeholder(tf.float32, [None, 784])
sample_z_func = encoder(x)

z = tf.placeholder(tf.float32, [None, z_dim])
sample_x_func = decoder(z)

# Encode
mean, var = sess.run(sample_z_func, feed_dict={x: x_test})
sample_z = mean

# Decode
sample_x = sess.run(sample_x_func, feed_dict={z: sample_z})


### to_csv ###
with open('/root/userspace/chap09/materials/submission.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(sample_x.reshape(-1, 28*28).tolist())

InternalError: Blas GEMM launch failed : a.shape=(100, 784), b.shape=(784, 200), m=100, n=200, k=784
	 [[Node: Encoder/dense/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_Placeholder_0_0/_1, Encoder/dense/kernel/read)]]
	 [[Node: Mean_1/_25 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_938_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Encoder/dense/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/pytho